# Webscraping Indeed for Job Titles with "Sustainability
## A Coding Temple Final Project
### Pray a little prayer for this to work

### Step 1: I created a conda virtual environment (finalproject_env) specific to this project and created a jupyter notebook that is connected to it.
For reference: https://www.youtube.com/watch?v=Ro9l0eapoJU
### Step 2: Start scraping.
I am using this as a reference: https://www.youtube.com/watch?v=g0QUvO8FfAE

In [2]:
import pandas as pd
import numpy as np
import lxml
import requests
from bs4 import BeautifulSoup as bs
import datetime
import time
from collections import defaultdict
import re

### Wait step back. Step 1.9: Study the URL and the DOM

- URL of interest: https://www.indeed.com/jobs?q=sustainability&l=California
- When you click on a specific job: https://www.indeed.com/jobs?q=sustainability&l=California&vjk=7bb3f8f043419fbd
    - there is a 'VJK' -- i think that means view job key?
- When you click the next page: https://www.indeed.com/jobs?q=sustainability&l=California&start=10
    - Increases by increments of 10
- When you view a specific job: https://www.indeed.com/viewjob?jk=178c02cc700910ee

All the job results are under 
 - div class="mosaic-zone" id='mosaic-zone-jobcards"
Followed by 
 - div id="mosaic-provider-jobcards" class="mosaic mosaic-provider-jobcards mosaic-provider-hydrated"
Each job card is an a tag that looks like this:
 - a id="sj_11cbe6c9847066e5" data-mobtk="1f5n5mhkkoci8801" data-jk="11cbe6c9847066e5" rel="nofollow" data-hide-spinner="true" class="tapItem result job_11cbe6c9847066e5 sponsoredJob resultWithShelf sponTapItem tapItem-noPadding desktop" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0ADzOeDESdYpV-bYAZsPPyr0JcDhLaY3k_3Q-ZKMNEAQZWeExh7cVECWgbhx4LTlu3PsybHKn7Y2qLo9GTuybO9xyWSDLKErIwmyQJmWkWQZUWD_h0NfMIINqc8T4-VB9UaOHG5q3tgMSVSUaWUQ-FQKMo6AT0zyAjZytcnIJQz6Up2pN5eKi_IeMl8kHDFaDYUakmWZAdALULBof_4qpq2XF3BtEn_DaUrO3DVardnEFRnTjU1QjnbIw1yGloyYxanujuXi0tiYYJI7gE38sii2Rw1AsI4b7BvixiaCas31nnJf2aFoO_0PHafxlk9ZKvQnY7l1BbU4m6dbH7XO-egXVY7BmZKTuQUOsfClbbgJlWJT8pLfnCdg7uTR4d2ipB-oXMGvcMKdA0yezDxoot-ozJHI987qXZBFnn8d4yzQ8W0j7g1lvAwQsYTYg5BaygKLJMkfrSGh6C3IKflxGleV9UUWdnNwsc=&amp;p=0&amp;fvj=1&amp;vjs=3" target="_blank"
    - div class="slider_container"
        - div class= "slider_list"
            - div class = "slider_item"
                - div class = "job_seen_beacon"
                    - table class = "jobCard_mainContent" - this is the top DEETS
                        - tbody --> tr --> tdclass = "resultContent
                            - <strong>JOB TITLE HERE</strong> div class="heading4 ..." --> h2 class=jobTitle -> get the title attribute in span
                            - <strong>COMPANY NAME HERE</strong> div class="heading6 ..." --> pre --> span class="companyName"
                            - <strong>LOCATION HERE</strong> ... --> div class="companyLocation"
                    - table class = jobCardShelfContainer"  - this is the bottom DEETS
                        - tbody --> tr class = "underShelfFooter" --> td --> divclass="heading6 tapItem-gutter result-footer"
                            - <strong>JOB SNIPPET HERE</strong> divclass = "job-snippet" --> ul --> li

Resource I used for modifying beautiful soup find_all/find paths, since the paths seems to have changed since the video was published: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree

In [12]:
#list of jobs
jj = []
counter = 0

num_jobs = 1000

for j in range(0, num_jobs, 10):
    
    print(f'Starting scrape for page {j//10 +1}/{num_jobs//10}.')
    
    #looping through the number of results page you want to parse, increments of 10
    position, location = 'sustainability', 'United+States'
    
    if j < num_jobs//4 or j > num_jobs*3//4:
        header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"}
    else:
        header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36"}
    
    y = requests.get(f'https://www.indeed.com/jobs?q={position}&l={location}&start={str(j)}', headers = header)
    
    print('status code is ', str(y.status_code))
    
    time.sleep(3)
    
    sp = bs(y.text, 'lxml')
    soup = sp.find('table',{"id":"pageContent"})
    #print(soup.find_all(name='div',attrs={"class":"jobsearch-SerpJobCard"}))
    #soup is the table with the results.
    #confirmed 12:07 am 5/15 that it returns a list with different div jobsearch-serpjobcard.
    #needed to run twice to get a title printed out. i think that's the carousel...
#     print(sp)
    for ii in soup.find_all(name='div',attrs={"class":"jobsearch-SerpJobCard"}):
        #for each div tag with the class = "jobsearch-SerpJobCard"
        job_title= ii.find('a',{"class":"jobtitle"})['title']
        company_name= ii.find('span',{"class":"company"}).text.strip()
        location= ii.find('span',{"class":"location"})
         
        if location:
            location=location.text.strip()
        else:
            location=ii.find('div',{"class":"location"})
            location=location.text.strip()
                
        #extracting the job key, which will lead us to the specific job's page
        k=bs(str(ii), 'lxml')
        jk=k.find(name='div', attrs={"class":"jobsearch-SerpJobCard"})
        jobkey=jk['data-jk']
    
        job_dict={'job_title': job_title,
                    'company_name': company_name,
                    'location': location,
                    'job_key': jobkey}
        
        counter += 1
        jj.append(job_dict)

print(f'Appended {counter} entries to jj list')

## 2:59 a.m., 05/15/21. Stopped working at line 14. Returns "Captcha Solve Page"
## 5:38 p.m, 05/15/21. Worked an hour ago. Scrapped 774 job listings fine, 
## but when extracting job descriptions from individual job pages, I noticed that it would return NaN for ~100? ish.
## I think it might be from the captcha. 
### My options -- do in batches of ~100? do that 10 times.
### -- Utilize user agents.
### -- i just need to scrape the job #s first. then run the scraper for job descriptions in batches, 
### which is probably where the user agents will come more in handy.

Starting scrape for page 0/99.
status code is  200
Starting scrape for page 1/99.
status code is  200
Starting scrape for page 2/99.
status code is  200
Starting scrape for page 3/99.
status code is  200
Starting scrape for page 4/99.
status code is  200
Starting scrape for page 5/99.
status code is  200
Starting scrape for page 6/99.
status code is  200
Starting scrape for page 7/99.
status code is  200
Starting scrape for page 8/99.
status code is  200
Starting scrape for page 9/99.
status code is  200
Starting scrape for page 10/99.
status code is  200
Starting scrape for page 11/99.
status code is  200
Starting scrape for page 12/99.
status code is  200
Starting scrape for page 13/99.
status code is  200
Starting scrape for page 14/99.
status code is  200
Starting scrape for page 15/99.
status code is  200
Starting scrape for page 16/99.
status code is  200
Starting scrape for page 17/99.
status code is  200
Starting scrape for page 18/99.
status code is  200
Starting scrape for pa

In [36]:
print(len(jj))
jj[:5]

1045


[{'job_title': 'Sustainability Intern',
  'company_name': 'Imperfect Foods',
  'location': 'Remote',
  'job_key': 'f5438de8a8e1629a'},
 {'job_title': 'Environmental Sustainability Internship',
  'company_name': 'Jellyfish',
  'location': 'Providence, RI',
  'job_key': '7075a3a7b5571059'},
 {'job_title': 'SUSTAINABILITY ANALYST',
  'company_name': 'LONGEVITY PARTNERS',
  'location': 'Austin, TX',
  'job_key': '2171570f88a3a65d'},
 {'job_title': 'Sustainability Innovation Consultant',
  'company_name': 'Amazon Web Services, Inc.',
  'location': 'Remote',
  'job_key': 'e862369e52ce2651'},
 {'job_title': 'Sustainability Associate',
  'company_name': 'Schneider Electric',
  'location': 'Louisville, KY',
  'job_key': '00a0e465fe8342c8'}]

In [34]:
## convert jj into a basic_jobs data frame. then convert to csv.
basic_jobs = pd.DataFrame(jj)
basic_jobs.to_csv('jobkeys.csv')

## Extract Data SUCCESS

### ITS WORKING!!! It seems like indeed does have some kind of carousel on their website, based on how I need to run the kernel a few times to get it to actually scrape the data and go through the for loop on line 18. 

### Now we have jj, which is the list with distinct job dictionaries

For reference, found another web scraping medium article: https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b

## Extract Data Part 2 - Job Descriptions

### Now we will use job_key to open a new job page and scrape that page for job_descriptions. 
### To use different user_agents, I used this resource:
https://www.youtube.com/watch?v=90t9WkQbQ2E&t=1s
https://developers.whatismybrowser.com/useragents/explore/software_name/chrome/2

## 5/15/21. User Agents not working? The job_keys were successfully extracted, but when I pull up each job page, I get a captcha. So , im gonna take a break and come back.

Also, Monster.com scraping seems more complex than indeed.com, because Monster.com uses React - Hash Routing. It's not its own separate page.

In [2]:
jj = pd.read_csv('jobkeys.csv')
jj.to_dict('list')
jk= (jj['job_key'])
print(type(jj))

<class 'pandas.core.frame.DataFrame'>


In [3]:
descriptions = {}
for i in range(641, len(jk)):

    #last job scraped was i = 657. start at i = 658 next. go 300.
    
#     if i < len(jj)//4 or i > len(jj)*3//4:
#         print(i)
#         header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
#     else:
#         header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}
    
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
    jp= requests.get(f"https://www.indeed.com/viewjob?jk={jk[i]}", headers = header)

    print(f"Request for {i}/{len(jk)} jobs")
    print(f"https://www.indeed.com/viewjob?jk={jk[i]}\n")
    
    jp_soup= bs(jp.text,'html.parser')
#     print(jp_soup)
    raw_desc = (jp_soup.find("div",{"class":"jobsearch-jobDescriptionText"}))
#     print(raw_desc)
    
    if raw_desc is None:
        print("NONE TYPE DETECTED")
        break
    
    try:
        descriptions[i] =(raw_desc.text.strip())
    except AttributeError:
        descriptions[i] = ""

#create list from descriptions dictionary values... this is what we will append to the jj dataframe.
descriptions_list = (list(descriptions.values()))

print(f'Extracted {len(descriptions_list)} job descriptions.')

Request for 641/1045 jobs
https://www.indeed.com/viewjob?jk=7ccdf0e26697d45d

Request for 642/1045 jobs
https://www.indeed.com/viewjob?jk=276a9389cb202861

Request for 643/1045 jobs
https://www.indeed.com/viewjob?jk=4268026343e3a4c1

Request for 644/1045 jobs
https://www.indeed.com/viewjob?jk=881b5bcbe9326589

Request for 645/1045 jobs
https://www.indeed.com/viewjob?jk=cc46fe639cfce9d2

Request for 646/1045 jobs
https://www.indeed.com/viewjob?jk=800d7cc3e9f91c0e

Request for 647/1045 jobs
https://www.indeed.com/viewjob?jk=81b9a3fdaedeaa13

Request for 648/1045 jobs
https://www.indeed.com/viewjob?jk=d7b394daa4a026e4

Request for 649/1045 jobs
https://www.indeed.com/viewjob?jk=99ec1a8e0fe9d7d8

Request for 650/1045 jobs
https://www.indeed.com/viewjob?jk=c43018565495fbf0

Request for 651/1045 jobs
https://www.indeed.com/viewjob?jk=463400a6db4dbe34

Request for 652/1045 jobs
https://www.indeed.com/viewjob?jk=5a293f636f07bc21

Request for 653/1045 jobs
https://www.indeed.com/viewjob?jk=3025

In [4]:
desc_jobs = pd.DataFrame(descriptions_list, columns=["job_descriptions"])
desc_jobs.to_csv('jobdesc_641_to_657.csv')

In [5]:
jj_test = jj
jj_test['job_num'] = jj_test['Unnamed: 0'].apply(lambda x: int(x))

In [6]:
desc_jobs['job_num']=[x for x in range(641,658)]
# desc_jobs.drop(['job_key'], inplace=True, axis = 1)
desc_jobs

,job_descriptions,job_num
0,FREE FLOW WINESJob DescriptionTitle: Director...,641
1,About UsSatco is a family-owned and operated m...,642
2,"PLANTA is building a team of inclusive, strate...",643
3,Relocation Assistance Offered Within Country\n...,644
4,"Job ID: 278719United Companies, is a CRH Compa...",645
5,We are seeking individuals that are highly ski...,646
6,Job Description:\n\nSummer Student Temp\nColla...,647
7,ERM is seeking a motivated Air Quality Staff C...,648
8,Overview:\n\n**This position will be: Hybrid: ...,649
9,Peraton Overview\n\nPeraton drives missions of...,650


In [7]:
jobs_onkey = jj_test.merge(desc_jobs, on = "job_num", how = "inner")
jobs_onkey
jobs_onkey.to_csv('jobs_increment_scrape_641_657.csv')

## Extracting Company Pages using Company Name

In [3]:
#Open the file you want to extract company_names from here

keys = pd.read_csv('missingcompanyinfo.csv')
keys

,Unnamed: 0,company_name
0,0,Gedeon GRC Consulting
1,1,California State University
2,2,"World Wide Technology, Inc."
3,3,EME Consulting Engineering Group
4,4,Indian Nations Council of Governments (INCOG)
...,...,...
99,99,Goby Inc
100,100,LG Vehicle Components
101,101,GreenPrint
102,102,Hines


In [42]:
## tester

html_doc = """
<li class="title"><span>The Dormouse's story</span><span> by Alice </span</li>
<ul class="css-ynb318-Box eu4oa1w0"><li data-testid="companyInfo-ceo" class="css-1m23au9-Box eu4oa1w0"><div class="css-1et0vwh-Box eu4oa1w0">CEO</div><div class="css-1cljifo-Box eu4oa1w0">Jamie Dimon</div></li><li data-testid="companyInfo-founded" class="css-1m23au9-Box eu4oa1w0"><div class="css-1et0vwh-Box eu4oa1w0">Founded</div><div class="css-1cljifo-Box eu4oa1w0">1799</div></li><li data-testid="companyInfo-employee" class="css-1m23au9-Box eu4oa1w0"><div class="css-1et0vwh-Box eu4oa1w0">Company size</div><div class="css-1cljifo-Box eu4oa1w0"><div class="css-4zutel-Box eu4oa1w0"><span><span>more than</span> 10,000</span></div></div></li><li data-testid="companyInfo-revenue" class="css-1m23au9-Box eu4oa1w0"><div class="css-1et0vwh-Box eu4oa1w0">Revenue</div><div class="css-1cljifo-Box eu4oa1w0"><div class="css-4zutel-Box eu4oa1w0"><span><span>more than</span> $10B (USD)</span></div></div></li><li data-testid="companyInfo-industry" class="css-1m23au9-Box eu4oa1w0"><div class="css-1et0vwh-Box eu4oa1w0">Industry</div><div class="css-1cljifo-Box eu4oa1w0">Financial Services</div></li></ul>
"""

soup = bs(html_doc, 'html.parser')

# print(soup.prettify())

for ii in soup.find_all(name="li"):
    if ii.get('data-testid') == 'companyInfo-ceo':
        print(re.sub('CEO', '', ii.text)) 
    elif ii.get('data-testid') == 'companyInfo-founded':
        print(re.sub('Founded', '', ii.text)) 
    elif ii.get('data-testid') == 'companyInfo-employee':
        print(re.sub('Company size', '', ii.text)) 
    elif ii.get('data-testid') == 'companyInfo-revenue':
        print(re.sub('Revenue', '',ii.text)) 
    elif ii.get('data-testid') == 'companyInfo-industry':
        print(re.sub('Industry', '', ii.text)) 


Jamie Dimon
1799
more than 10,000
more than $10B (USD)
Financial Services


In [4]:
cc = []
counter = 0
pagenotfound = 0
noinfo = 0

for c in range(len(keys)):
    # last scrape was c=352. start at c=353
    cd = {'ceo': '-1',
          'founded': '-1',
          'size': '-1',
          'revenue': '-1',
          'industry': '-1'       
         }
    
    cname = keys['company_name'][c]
    
    #formatting cname for URL
    #Extract name before comma
    fcname = re.split(r',+',cname)
    
    #replace space with -
    fcname = re.sub(r'\s','-',fcname[0])
    
    cr = requests.get(f'https://www.indeed.com/cmp/{fcname}')
    #404 is page not found
    #200 is good
    if c % 2 == 0 and c % 5 == 0 and c < 10:
        time.sleep(7)
    elif c % 2 != 0 and c % 5 == 0 and c < 10:
        time.sleep(4)
    elif c% 3 != 0 and c < 10:
        time.sleep(3)
    elif c % 2 == 0 and c % 5 == 0 and c >= 10:
        time.sleep(2)
    elif c % 2 != 0 and c % 5 == 0 and c >= 10:
        time.sleep(9)
    elif c % 3 != 0 and c >= 10:
        time.sleep(5)
    
    time.sleep(5)
    
    if cr.status_code == 200:
        sp = bs(cr.text, 'lxml')
        soup = sp.find('ul', {"class": 'eu4oa1w0'})
        
        print(f'{c} {fcname} REQUEST SUCCESSFUL')
        
        
        if soup is None:
            print("\tNONE TYPE DETECTED")
            noinfo += 1
            cc.append(cd)
            continue
        
        for ii in soup.find_all(name="li"):
            if ii.get('data-testid') == 'companyInfo-ceo':
                cd['ceo'] = (re.sub('CEO', '', ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-founded':
                cd['founded'] = (re.sub('Founded', '', ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-employee':
                cd['size'] = (re.sub('Company size', '', ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-revenue':
                cd['revenue'] = (re.sub('Revenue', '',ii.text)) 
            elif ii.get('data-testid') == 'companyInfo-industry':
                cd['industry'] = (re.sub('Industry', '', ii.text)) 
        print("\tInfo Extract SUCCESS")
            
    else:
        print(f'{c} {fcname} PAGE NOT FOUND')
        pagenotfound +=1
    
    
    counter += 1
    cc.append(cd)
    
print(f'Appended {counter} companies to cc list')
print(f'Page Not Found for {pagenotfound} companies') # 8 comp (353`641),, 
print(f'No info for {noinfo} companies')  #23 (353`641),, 

0 Gedeon-GRC-Consulting REQUEST SUCCESSFUL
	Info Extract SUCCESS
1 California-State-University REQUEST SUCCESSFUL
	Info Extract SUCCESS
2 World-Wide-Technology REQUEST SUCCESSFUL
	Info Extract SUCCESS
3 EME-Consulting-Engineering-Group REQUEST SUCCESSFUL
	NONE TYPE DETECTED
4 Indian-Nations-Council-of-Governments-(INCOG) REQUEST SUCCESSFUL
	NONE TYPE DETECTED
5 Universal-Leaf-Tobacco-Co.-Inc. REQUEST SUCCESSFUL
	Info Extract SUCCESS
6 Studio-School---Los-Angeles PAGE NOT FOUND
7 Epsten-Group REQUEST SUCCESSFUL
	Info Extract SUCCESS
8 Home-Innovation-Research-Labs REQUEST SUCCESSFUL
	Info Extract SUCCESS
9 Onward-Energy REQUEST SUCCESSFUL
	NONE TYPE DETECTED
10 Shenandoah-Valley-Organic REQUEST SUCCESSFUL
	Info Extract SUCCESS
11 ima-Design REQUEST SUCCESSFUL
	NONE TYPE DETECTED
12 City-of-Naples REQUEST SUCCESSFUL
	Info Extract SUCCESS
13 Benson-Hill REQUEST SUCCESSFUL
	Info Extract SUCCESS
14 Cooper-Carry REQUEST SUCCESSFUL
	Info Extract SUCCESS
15 US-Surface-Transportation-Board REQU

In [11]:
cd=pd.DataFrame(cc)
cd = cd.join(keys)
cd.to_csv('missingcompanyinfo_filled.csv')

In [24]:
#test to see if we reached captcha yet.
fname="AECOM"
cr = requests.get(f'https://www.indeed.com/cmp/{fname}')

time.sleep(3)
    
sp = bs(cr.text, 'lxml')
soup = sp.find('ul', {"class": 'eu4oa1w0'})
print(soup)

<ul class="css-ynb318-Box eu4oa1w0"><style data-emotion="css 1m23au9-Box">.css-1m23au9-Box{box-sizing:border-box;margin:0;min-width:0;display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;background-color:#ffffff;border:1px solid #f3f2f1;border-radius:0.5rem;padding:16px;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;position:relative;-webkit-flex:0 0 120px;-ms-flex:0 0 120px;flex:0 0 120px;margin:8px;box-sizing:border-box;word-wrap:break-word;}@supports (display: grid){.css-1m23au9-Box{margin:0;-webkit-flex:0 0 147px;-ms-flex:0 0 147px;flex:0 0 147px;}}</style><li class="css-1m23au9-Box eu4oa1w0" data-testid="companyInfo-ceo"><style data-emotion="css 1et0vwh-Box">.css-1et0vwh-Box{box-sizing:border-box;margin:0;min-width:0;margin-bottom:16px;font-size:14px;font-weight:700;display:block;}</style><div class="css-1et0vwh-Box eu4oa1w0">CEO</div><style data-emotion="css 1cljifo-Box">.css-1cljifo-Box{box-sizing:border-box;margin:0;min-width:

In [51]:
cnames = pd.DataFrame(keys[['Unnamed: 0.1','company_name']])
# cnames['keyn'] = [x for x in range(641)]
# list(cnames['company_name'][0:190])
cnames

,Unnamed: 0.1,company_name
0,641,Free Flow Wines
1,642,"Satco, Inc."
2,643,Planta
3,644,Tom's of Maine
4,645,CRH
5,646,Brodie Builders
6,647,3M
7,648,ERM Group
8,649,City of Boston
9,650,Peraton


#### Helpful resource for pandas merging...
https://stackoverflow.com/questions/53645882/pandas-merging-101/

In [52]:
comp_info = pd.DataFrame(cc)
comp_info['keyn'] = [x for x in range(641,658)]
comp_info
cc_info = cnames.merge(comp_info, how = "inner", left_on ='Unnamed: 0.1', right_on = 'keyn')
cc_info

,Unnamed: 0.1,company_name,ceo,founded,size,revenue,industry,keyn
0,641,Free Flow Wines,-1,2009,less than 10,$5M to $25M (USD),Food & Beverage Manufacturing,641
1,642,"Satco, Inc.",-1,-1,-1,-1,Banking & Lending,642
2,643,Planta,-1,-1,-1,-1,-1,643
3,644,Tom's of Maine,-1,2006,51 to 200,$25M to $100M (USD),-1,644
4,645,CRH,-1,-1,-1,-1,"Construction, Repair & Facilities Services",645
5,646,Brodie Builders,-1,-1,-1,-1,Architectural & Engineering Services,646
6,647,3M,-1,-1,-1,-1,-1,647
7,648,ERM Group,-1,1971,"1001 to 5,000",$100M to $500M (USD),Management & Consulting,648
8,649,City of Boston,Marty Walsh,1630,"more than 10,000",-1,Government & Public Administration,649
9,650,Peraton,Stu Shea,2017,"more than 10,000",$5B to $10B (USD),Aerospace & Defense,650


In [53]:
cc_info.to_csv('companyinfo_641_657.csv')

In [7]:
combjobs = pd.read_csv('combined_jobs_0_640.csv')
compinfo = pd.read_csv('companyinfo_0_33.csv')
all_info_0_33 = combjobs.merge(compinfo, how = "left", on='company_name')
all_info_0_33.drop('Unnamed: 0_y', inplace= True, axis=1)
all_info_0_33.drop('Unnamed: 0_x', inplace= True, axis=1)
all_info_0_33.to_csv('allinfo_0_33.csv')

### First increment transform

In [12]:
#out dated. this was for first increment transform

# jj is already a dataframe, after reading the job keys csv
# basic_jobs = pd.DataFrame(jj)
# desc_jobs = pd.DataFrame(descriptions_list, columns=["job_descriptions"])
jobs_ = pd.concat([jj, desc_jobs], axis=1)
jobs_['job_descriptions']
jobs_.to_csv('jobs_increment_scrape_336.csv')